In [2]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.animation
%matplotlib ipympl
import numpy as np
import pandas as pd

from scipy.spatial import KDTree, cKDTree
from scipy.ndimage import uniform_filter1d
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit

import time
import joblib
from tqdm import tqdm
import trackpy as tp
from numba import njit, prange

from yupi import Trajectory
import yupi.graphics as yg
import yupi.stats as ys

from utility import get_imsd, get_imsd_windowed, get_emsd, get_emsd_windowed, fit_hist, MB_2D,\
                    normal_distr, lorentzian_distr, get_trajs, speed_windowed, theta_windowed, \
                    get_smooth_trajs, get_velocities

In [3]:
traj_verb = "stardist"

if traj_verb == "trackpy": 
    rawTrajs = pd.read_parquet("../tracking/results/tracking_data/trackpy_pre_merge.parquet")
    nDrops = int(len(rawTrajs.loc[rawTrajs.frame==0]))
    red_particle_idx = 17
    # Trajectory Smoothing: using a Savgol Filter in order to drop the noise due to the tracking procedure
    smoothTrajs = get_smooth_trajs(rawTrajs, nDrops, 30, 2)

    res_path = "results"
    analysis_data_path = "analysis_data"

elif traj_verb == "hough":
    rawTrajs = pd.read_parquet("../tracking/results/tracking_data//hough/linked_pre_merge.parquet")#pd.read_parquet("../tracking/results/tracking_data/tracking_hough_trackpy_linking.parquet")
    nDrops = int(len(rawTrajs.loc[rawTrajs.frame==0]))
    red_particle_idx = 17
    # Trajectory Smoothing: using a Savgol Filter in order to drop the noise due to the tracking procedure
    smoothTrajs = get_smooth_trajs(rawTrajs, nDrops, 30, 2)
    
    res_path = "hough_results"
    analysis_data_path = "hough_analysis_data"

elif traj_verb == "stardist":
    rawTrajs = pd.read_parquet("../tracking/stardist_res/sharp/df_linked.parquet")

    #rawTrajs = pd.read_parquet("../tracking/stardist_res/noblur/df_linked.parquet") -- #red_particle_idx = 10
    nDrops = int(len(rawTrajs.loc[rawTrajs.frame==0]))
    # Trajectory Smoothing: using a Savgol Filter in order to drop the noise due to the tracking procedure
    smoothTrajs = get_smooth_trajs(rawTrajs, nDrops, 30, 2)
    red_particle_idx = 8

    res_path = "stardist_results" 
    analysis_data_path = "stardist_analysis_data"
else:
    raise ValueError("traj_verb must be either 'trackpy' or 'hough'")


colors = rawTrajs.loc[rawTrajs.frame == 0, 'color'].values
nFrames = int(max(rawTrajs.frame) + 1)
fps = 10
print(f"Number of Droplets: {nDrops}")
print(f"Number of Frames: {nFrames} at {fps} fps --> {nFrames/10:.2f} s")

# WINDOWED ANALYSIS PARAMETERS
window = 3200 # 320 s
stride = 100 # 10 s
print("Windowed analysis args:")

startFrames = np.arange(0, nFrames-window, stride, dtype=int)
endFrames = startFrames + window
nSteps = len(startFrames)
print(f"window of {window/10} s, stride of {stride/10} s --> {nSteps} steps")

# step 10 with a 10 fps video --> 1 s
units = "px/s"
default_kwargs_blue = {"color": "#00FFFF", "ec": (0, 0, 0, 0.6), "density": True}
default_kwargs_red = {"color": "#EE4B2B", "ec": (0, 0, 0, 0.6), "density": True}

Number of Droplets: 50
Number of Frames: 30000 at 10 fps --> 3000.00 s
Windowed analysis args:
window of 320.0 s, stride of 10.0 s --> 268 steps


# MSD

In [3]:
show_verb = False
run_windowed_analysis = True
plot_verb = True
animated_plot_verb = True
save_verb = True

print(f"MSD Analysis: trajs_verb = {traj_verb}, show_verb = {show_verb}, run_windowed_analysis = {run_windowed_analysis}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/msd.ipynb
#%run ./analysis_modules/msd.py

MSD Analysis: trajs_verb = stardist, show_verb = False, run_windowed_analysis = True, animated_plot_verb = True
Global IMSD
Global EMSD
Raw trajs - Blue Particles: a = 1.178 ± 0.002, K = 61.499 ± 0.431
Raw trajs - Red Particle: a = 0.763 ± 0.001, K = 1475.92 ± 8.618
Smooth trajs - Blue Particles: a = 1.18 ± 0.002, K = 61.038 ± 0.435
Smooth trajs - Red Particle: a = 0.765 ± 0.001, K = 1455.78 ± 8.733
Windowed IMSD


100%|██████████| 268/268 [04:46<00:00,  1.07s/it]
14:59:41 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
14:59:41 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/mean_squared_displacement/windowed_analysis/IMSD_wind_raw.mp4
15:00:24 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
15:00:24 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/mean_squared_displacement/windowed_analysis/IMSD_wind_smooth.mp4


Windowed EMSD


15:01:16 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
15:01:16 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/mean_squared_displacement/windowed_analysis/EMSD_wind_raw.mp4
15:01:48 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
15:01:48 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/mean_squared_displacement/windowed_analysis/EMSD_wind_smooth.mp4


# VELOCITY DISTRIBUTION

$$ \huge p(v) = \frac{v}{\sigma^2} e^{-\frac{v^2}{2\sigma^2}} $$

In [4]:
show_verb = False
plot_verb = True
animated_plot_verb = True
save_verb = True

v_step = 10

print(f"Speed Analysis: trajs_verb = {traj_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/velocity_distr.ipynb

Speed Analysis: trajs_verb = stardist, show_verb = False, animated_plot_verb = True

 Global speed distribution analysis
Raw Trajectories
Blue Particles σ: 2.842059051868812 ± 0.019462201664093782
Red Particle σ: 8.527733950350948 ± 0.175593143474466
Smooth Trajectories
Blue Particles σ: 2.6965480913153352 ± 0.018441636684297272
Red Particle σ: 8.595841091784294 ± 0.17696891091328149

 Windowed speed distribution Analysis


100%|██████████| 268/268 [01:03<00:00,  4.20it/s]
15:04:38 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
15:04:38 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/speed_distribution/speed_raw_wind.mp4
15:06:45 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
15:06:45 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/speed_distribution/speed_smooth_wind.mp4


# TURNING ANGLES DISTRIBUTIONS

In [11]:
show_verb = False
run_windowed_analysis = True
plot_verb = True
animated_plot_verb = True
save_verb = True

# choose the distribution to fit
distribution_str = "lorentzian"

if distribution_str == "gaussian":
    distribution = normal_distr
elif distribution_str == "lorentzian":
    distribution = lorentzian_distr
    distribution_str = "lorentzian"
else:
    raise ValueError("distribution_str must be either 'gaussian' or 'lorentzian'")

print(f"Turning Angles Analysis: trajs_verb = {traj_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/turning_angles.ipynb

Turning Angles Analysis: trajs_verb = stardist, show_verb = False, animated_plot_verb = True

 Global turning angles analysis
Raw Trajectories
Blue Particles σ: 0.27552311686634195 ± 0.0031321569279026697, μ: -0.0001749427467774336 ± 0.0031312785630483696
Red Particle σ: 0.18034505358162833 ± 0.002232638268296026, μ: 0.007301939331799477 ± 0.0022323670065862856
Smooth Trajectories
Blue Particles σ: 0.07207792721412327 ± 0.0005473298139118742, μ: 0.00011912635376943489 ± 0.00045339684518952
Red Particle σ: 0.054037099393034464 ± 0.0007573700968624189, μ: -0.001883291890670179 ± 0.00045121658450882914

 Windowed turning angles analysis


  1%|          | 3/268 [00:00<01:19,  3.33it/s]


KeyboardInterrupt: 

KeyboardInterrupt: 

# VELOCITY AUTOCORRELATION

In [6]:
show_verb = False
run_windowed_analysis = True
plot_verb = True
animated_plot_verb = True
save_verb = True
overwrite = False

print(f"Velocity Autocorrelation Analysis: show_verb = {show_verb}, run_windowed_analysis = {run_windowed_analysis}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/velocity_autocorrelation.ipynb

Velocity Autocorrelation Analysis: show_verb = False, run_windowed_analysis = True, animated_plot_verb = True
Global Velocity Autocovariance Function
Windowed analysis


100%|██████████| 268/268 [37:32<00:00,  8.40s/it] 
16:27:35 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
16:27:35 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/velocity_autocovariance/vacf_wind_raw.mp4
16:27:55 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
16:27:55 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/velocity_autocovariance/vacf_wind_smooth.mp4


# RADIAL DISTRIBUTON FUNCTION

In [4]:
run_analysis_verb = True
show_verb = False
animated_plot_verb = True
save_verb = True

print(f"Radial Distribution Function Analysis: {traj_verb}, run_analysis_verb = {run_analysis_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/radial_distribution_function.ipynb

Radial Distribution Function Analysis: stardist, run_analysis_verb = True, show_verb = False, animated_plot_verb = True
RDF - Raw Trajectories
raw


  1%|          | 255/30000 [00:45<1:38:24,  5.04it/s]

KeyboardInterrupt: 

KeyboardInterrupt: 

# 3 BODY STRUCTURE METRIC

In [ ]:
run_analysis_verb = False
show_verb = False
animated_plot_verb = False

%run ./analysis_modules/3body_analysis.ipynb